# Demo for Trading Backtesting

I have a previous project which is helpful to simply have something as a simple single stock backtester.

It is important to note that this does not include a universe of tickers or have shorting mechanics (*just has long positions and take profit/stop loss logic*)

In [ ]:

import pandas as pd 
from enum import Enum
from datetime import datetime
import uuid 
import logging

logging.basicConfig(
    filename='debug.log',
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.DEBUG
)


prices = pd.read_csv('Nvidia2024-2014.csv')
prices.rename(columns={'Close/Last': 'Close'}, inplace=True)

usd_to_gbp_rate = 0.76

price_columns = ['Close', 'Open', 'High', 'Low']

prices['Date'] = pd.to_datetime(prices['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')


for col in price_columns:
    prices[col] = prices[col].astype(str).replace({'\\$': ''}, regex=True).astype(float)

for col in price_columns:
    prices[col] = prices[col] * usd_to_gbp_rate
prices.head()

## Initial framework for dummy trades

In [ ]:
INITIAL_WALLET_CASH = 5_000_000
STANDARD_LOT = 100000

class Side(Enum):
    BUY = 'BUY'
    SELL = 'SELL'
    
class Currency(Enum):
    GBP = 'GBP'
    USD = 'USD'
    
class Position:
    def __init__(self,
                 date:datetime,
                 size:int,
                 price,
                 currency:Currency,
                 side:Side):
        
        self._id = str(uuid.uuid4())
        self._date = date
        self._currency = currency
        self._price = price
        self._size = size
        self._side = side
        
class Order:
    def __init__(self,
                 position:Position):
        self._id = str(uuid.uuid4())
        self._position = position
    
    def describe(self):
        return f"<Order id: {self._id} position: {self._position}>"

class Portfolio:
    
    def __init__(self):
        self.cash_wallet = INITIAL_WALLET_CASH  # wallet is in GBP
        self.current_positions = {}
        self.closed_positions = {}
    
    def add_position(self,position):
        self.current_positions[position._id] = position
        self.deduct_from_wallet(self.calculate_deduction(position))
                
    def sell_position(self,position:Position):
        self.current_positions.pop(position._id)
        self.closed_positions[position._id] = position
    
    def get_wallet_value(self):
        return self.cash_wallet
    
    def deduct_from_wallet(self,amount):
        self.cash_wallet -= amount
        
    def add_to_wallet(self,amount):
        self.cash_wallet += amount
        
    def calculate_fees(self,position:Position):
        fee = position._size * 2 #broker fee is £2 per standard lot 
        logging.info(f"FEE CHARGED: £{fee}")
        return fee 
    
    def calculate_total_cost_in_gbp(self,position:Position):
        if(position._currency == Currency.GBP):
            return position._price * (position._size * STANDARD_LOT)
        # elif(position._currency == Currency.USD):
        #     return position._price * current_gbp_price  * (position._size * 100000) # x100000 for standard lot

    def calculate_deduction(self,position):
        return self.calculate_total_cost_in_gbp(position) + self.calculate_fees(position)
    

## Take Profit and Stop Loss mechanics

In [ ]:
def check_take_profit(FIXED_LOT_SIZE, A_REALLY_BIG_NUMBER, orders, tp_prices, row, currentP, date):
    if (tp_prices != []):
        cTP = min(tp_prices)
            
        while(currentP>=cTP):
            sell_position = Position(date,FIXED_LOT_SIZE,row.Open,Currency.GBP,Side.SELL)
            sell_order = Order(sell_position)
            orders[sell_order._id] = sell_order
                
            tp_prices.pop(tp_prices.index(cTP))
            try:
                cTP = min(tp_prices)
            except ValueError:
                cTP = A_REALLY_BIG_NUMBER

def check_stop_loss(FIXED_LOT_SIZE, A_REALLY_SMALL_NUMBER, orders, sl_prices, row, currentP, date):
    if (sl_prices != []):
        cSL = max(sl_prices)

        while(currentP<=cSL):
            sell_position = Position(date,FIXED_LOT_SIZE,row.Open,Currency.GBP,Side.SELL)
            sell_order = Order(sell_position)
            orders[sell_order._id] = sell_order

            sl_prices.pop(sl_prices.index(cSL))
            try:
                cSL = max(sl_prices)

            except ValueError:
                cSL = A_REALLY_SMALL_NUMBER


## Setup indicators

Can add more strategies or remove ones that might not be impactful

In [ ]:
def setup_indicators(rsi_level, bb_mult, macd_period, data):
     # calculate Bollinger Bands
    data['MA20'] = data['Close'].rolling(window=20).mean()
    data['MA20_std'] = data['Close'].rolling(window=20).std()
    data['Upper_band'] = data['MA20'] + (data['MA20_std'] * bb_mult)
    data['Lower_band'] = data['MA20'] - (data['MA20_std'] * bb_mult)
    
    # calculate RSI
    period = 14
    delta = data['Open'].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    AVG_Gain = up.rolling(window=period).mean()
    AVG_Loss = abs(down.rolling(window=period).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    data['RSI'] = RSI

     # calculate Moving Average Convergence Divergence (MACD)
    period_short = macd_period[0]
    period_long = macd_period[1]
    data['EMA_short'] = sum(data['Close'][len(data)-period_short:])/period_short
    data['EMA_long'] = sum(data['Close'][len(data)-period_long:])/period_long
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal'] = sum(data['MACD'][len(data)-9:])/9

    # calculate Stochastic Oscillator
    period = 14
    data['L14'] = data['Low'].rolling(window=period).min()
    data['H14'] = data['High'].rolling(window=period).max()
    data['%K'] = (data['Close'] - data['L14']) / (data['H14'] - data['L14']) * 100
    data['%D'] = data['%K'].rolling(window=3).mean()
    return data


## Strategy function

edit the below cells to change the strategy, at the moment just use all the indicators to make a buy or sell order.
can add more "*special cases*" for potential trading opportunities missed

In [ ]:
FIXED_LOT_SIZE = 10
A_REALLY_BIG_NUMBER = 100000000
A_REALLY_SMALL_NUMBER = 0.0000000000001

def check_to_sell(orders, row, current_price, date, rsi_level):
    if(row["Close"] > row["Upper_band"] or row["RSI"] > rsi_level or row["MACD"] < row["Signal"] and (row["%K"] > 80 and row["%D"] > 80 and row["%K"]<row["%D"])):
        sell_position = Position(date,FIXED_LOT_SIZE,current_price,Currency.GBP,Side.SELL)
        sell_order = Order(sell_position)
        orders[sell_order._id] = sell_order

def check_to_buy(orders, tp_prices, sl_prices, row, current_price, date, rsi_level):
    if((row["Close"] < row["Lower_band"] and row["RSI"] < rsi_level) or row["MACD"] > row["Signal"] and (row["%K"] < 20 and row["%D"] < 20 and row["%K"]<row["%D"])):
        buy_position = Position(date,FIXED_LOT_SIZE,current_price,Currency.GBP,Side.BUY)
        buy_order = Order(buy_position)
        orders[buy_order._id] = buy_order
        tp_prices.append(row["Upper_band"])
        sl_prices.append(row["Lower_band"])


In [ ]:
def strategy(rsi_level, bb_mult, macd_period, data):
    
    orders = {}
    tp_prices=[]
    sl_prices=[]
    
    data = setup_indicators(rsi_level, bb_mult, macd_period, data)
    
    for index,row in data.iterrows():
        current_price = row["Open"]+row["Close"]/2

        date = datetime.strptime(str(row["Date"]),'%Y-%m-%d').date()
        
        check_take_profit(FIXED_LOT_SIZE, A_REALLY_BIG_NUMBER, orders, tp_prices, row, current_price, date)
        check_stop_loss(FIXED_LOT_SIZE, A_REALLY_SMALL_NUMBER, orders, sl_prices, row, current_price, date)

        
        check_to_buy(orders, tp_prices, sl_prices, row, current_price, date, rsi_level[0])
            

        check_to_sell(orders, row, current_price, date, rsi_level[1])

    return orders

## Backtest function

At the moment this just goes through all the orders made from the strategy function and simulates how profitiable it would be if the orders were made.

In [ ]:
def backtest_strategy(rsi_level, bb_mult, macd_period, data):
    
    portfolio = Portfolio()
    order_queue = strategy(rsi_level, bb_mult, macd_period, data)
    
    for index, order in order_queue.items():

        logging.info(f"Side is {order._position._side}")
        logging.info(f"Current P/L: {portfolio.cash_wallet - INITIAL_WALLET_CASH}")

        # BUY orders
        if order._position._side == Side.BUY:
            if portfolio.calculate_total_cost_in_gbp(order._position) <= portfolio.cash_wallet:
                portfolio.add_position(order._position)  # Deduct cash
                logging.info(f"BUY ORDER FILLED: (ID) {str(order._id).upper()}")
            else: 
                logging.info("INSUFFICIENT CASH FOR BUY ORDER")
        
        # SELL orders
        elif order._position._side == Side.SELL:
            # Find the earliest matching BUY position (by size) to sell
            matching_position = next(
                (pos for pos in portfolio.current_positions.values()
                 if pos._side == Side.BUY and pos._size == order._position._size),
                None
            )
            
            if matching_position:
                sale_earning = order._position._price * (order._position._size * STANDARD_LOT)
                buy_cost = matching_position._price * (matching_position._size * STANDARD_LOT)
                portfolio.add_to_wallet(sale_earning)
                portfolio.sell_position(matching_position)  # Remove the matching BUY position                
                portfolio.add_to_wallet(buy_cost)
                fee = portfolio.calculate_fees(order._position)
                portfolio.deduct_from_wallet(fee)

                logging.info(f"SELL ORDER FILLED: (ID) {str(order._id).upper()}")
            else:
                logging.info("NO MATCHING BUY POSITION FOUND")
    
    # After all orders processed
    print("ALL ORDERS HAVE BEEN FILLED.")
    print("BOOK CASH LEVEL:", ("£{:,.2f}".format(portfolio.cash_wallet)))

    pnl = portfolio.cash_wallet - INITIAL_WALLET_CASH

    if pnl < 0:
        print("TOTAL LOSS:", ("£{:,.2f}".format(abs(pnl))))
    else:
        print("TOTAL PROFIT:", ("£{:,.2f}".format(pnl)))

    roi = (pnl / INITIAL_WALLET_CASH) * 100
    print("RATE ON INVESTMENT: {:.2f}%".format(roi))

    return pnl


## Function to test best params for strategy

In [ ]:
import itertools



def test_strategy(data):
    params_for_testing = define_params()
    rsi_levels,bollinger_mults,macd_periods = params_for_testing[0],params_for_testing[1],params_for_testing[2]

    # Run grid search for optimization
    best_pnl = float('-inf')
    best_params = None

    for rsi_level, bb_mult, macd_period in itertools.product(rsi_levels, bollinger_mults, macd_periods):
        pnl = backtest_strategy(rsi_level, bb_mult, macd_period, data)
        if pnl > best_pnl:
            best_pnl = pnl
            best_params = (rsi_level, bb_mult, macd_period)

    print("Best PnL:", best_pnl)
    roi = (best_pnl / INITIAL_WALLET_CASH) * 100
    print("RATE ON INVESTMENT: {:.2f}%".format(roi))
    print(f"Optimal Parameters:\nRSI: low - {best_params[0][0]} high - {best_params[0][1]}\nBollinger Bands STD: {best_params[1]}\nMACD Time Period: short - {best_params[2][0]} long - {best_params[2][1]}")
    print(f"rsi_levels {rsi_levels}, bol {bollinger_mults} and macd {macd_periods}")

def define_params():
    rsi_levels = [(low, high) for low in range(25, 40, 5) for high in range(65, 75, 5)]
    bollinger_mults = [x / 100 for x in range(150, 201, 25)]  # 1.50, 1.75, 2.00, 2.25, 2.50
    macd_periods = [(short, long) for short in range(10, 13) for long in range(24, 27)]  # Short: 10-11, Long: 12-14

    num_combinations = len(rsi_levels) * len(bollinger_mults) * len(macd_periods)

    print(f"Number of RSI: {len(rsi_levels)}\nNumber of bollinger mults: {len(bollinger_mults)}\nNumber of MACD periods: {len(macd_periods)}\nNumber of combinations: {num_combinations}")

    return (rsi_levels,bollinger_mults,macd_periods)

## WARNING

**THIS IS A BRUTEFORCE APPROACH AND SO RESULTS IN A LOT OF COMBINATIONS**

You can refine the search space however, I saw this resulted in less trades being made netting 0% ROI. You can also change the ranges of the parameters being tested

Run the function below to check the number of combinations and to find what can be reduced

In [ ]:
define_params()

## BEST STRATEGY?

RSI: low - 35 high - 65 <br>
Bollinger Bands STD: 1.5 <br>
MACD Time Period: short - 10 long - 24

In [ ]:
backtest_strategy((35,65),1.5,(10,24),prices)

Something to note: at the moment the orders price is just the price the order was put in for so we are assuming each order is fulfilled. Obviously in a real market if we put in an order to sell something for 1,000,000 it wouldn't be fulfilled. The strategy just enters orders using the average of the opening and the close prices, (again not very realistic so requires improvement)